# Mini Project 2A

<b>Project Title: </b>Text Utilities <br>
<b>Domain: </b>Natural Language Processing

<b>Team Members:</b>
1. Mitali Rawat (Roll No. 21)
2. Radha Vishwakarma (Roll No. 46)
3. Om Achrekar (Roll No. 54)

<b>Problem Statement</b>: To create a web application that will help user to summerize, paraphrase the given text, and will also be able to correct the grammatical mistakes.

## Grammar correction using T5 transformer:

#### Done By: Om Achrekar

Hugging Face Hub Link: https://huggingface.co/achrekarom/grammar_correction

In [8]:
pip install happytransformer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [9]:
from happytransformer import HappyTextToText

happy_tt = HappyTextToText("T5", "t5-base")

10/08/2022 15:00:28 - INFO - happytransformer.happy_transformer -   Using model: cpu


In [10]:
from datasets import load_dataset

In [11]:
train_dataset = load_dataset("jfleg", split='validation[:]')

eval_dataset = load_dataset("jfleg", split='test[:]')

/opt/anaconda3/envs/project/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


In [12]:
for case in train_dataset["corrections"][:2]:
  print(case)
  print(case[0])

['So I think we would not be alive if our ancestors did not develop sciences and technologies . ', 'So I think we could not live if older people did not develop science and technologies . ', 'So I think we can not live if old people could not find science and technologies and they did not develop . ', 'So I think we can not live if old people can not find the science and technology that has not been developed . ']
So I think we would not be alive if our ancestors did not develop sciences and technologies . 
['Not for use with a car . ', 'Do not use in the car . ', 'Car not for use . ', 'Can not use the car . ']
Not for use with a car . 


In [13]:
import csv

def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for case in dataset:
     	    # Adding the task's prefix to input 
            input_text = "grammar: " + case["sentence"]
            for correction in case["corrections"]:
                # a few of the cases contain blank strings. 
                if input_text and correction:
                    writter.writerow([input_text, correction])
                    


generate_csv("train.csv", train_dataset)
generate_csv("eval.csv", eval_dataset)

In [16]:
 before_result = happy_tt.eval("eval.csv")

10/08/2022 15:51:01 - INFO - happytransformer.happy_transformer -   Preprocessing evaluating data...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /Users/omachrekar/.cache/huggingface/datasets/csv/default-ef6e178e80894712/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

/opt/anaconda3/envs/project/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
***** Running Evaluation *****
  Num examples = 2988
  Batch size = 1
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [17]:
print("Before loss:", before_result.loss)

Before loss: 1.2803850173950195


In [18]:
from happytransformer import TTTrainArgs

args = TTTrainArgs(batch_size=8)
happy_tt.train("train.csv", args=args)

10/08/2022 16:16:40 - INFO - happytransformer.happy_transformer -   Preprocessing training data...


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /Users/omachrekar/.cache/huggingface/datasets/csv/default-1bee5995f6642db1/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

/opt/anaconda3/envs/project/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
10/08/2022 16:16:44 - INFO - happytransformer.happy_transformer -   Training...
PyTorch: setting up devices
/opt/anaconda3/envs/project/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3016
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, di

Step,Training Loss
500,0.578400
1000,0.443900




Training completed. Do not forget to share your model on huggingface.co/models =)




In [19]:
before_loss = happy_tt.eval("eval.csv")

print("After loss: ", before_loss.loss)

10/08/2022 20:20:58 - INFO - happytransformer.happy_transformer -   Preprocessing evaluating data...


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

PyTorch: setting up devices
***** Running Evaluation *****
  Num examples = 2988
  Batch size = 1


After loss:  0.4503001570701599


In [20]:
from happytransformer import TTSettings

beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=20)

In [21]:
user_ip = input("Enter a sentence: ")
pass_this = "grammar:" + user_ip
result = happy_tt.generate_text(pass_this, args=beam_settings)
print(result.text)

Enter a sentence: I has one book
I have one book .


In [22]:
user_ip = input("Enter a sentence: ")
pass_this = "grammar:" + user_ip
result = happy_tt.generate_text(pass_this, args=beam_settings)
print(result.text)

Enter a sentence: He are very bright student
He are a very bright student .


In [23]:
user_ip = input("Enter a sentence: ")
pass_this = "grammar:" + user_ip
result = happy_tt.generate_text(pass_this, args=beam_settings)
print(result.text)

Enter a sentence: Radha have studied for exams
Rahha has studied for exams .


In [24]:
user_ip = input("Enter a sentence: ")
pass_this = "grammar:" + user_ip
result = happy_tt.generate_text(pass_this, args=beam_settings)
print(result.text)

Enter a sentence: Do not throow garbage on rod.
Do not throw garbage on a rod.


In [25]:
user_ip = input("Enter a sentence: ")
pass_this = "grammar:" + user_ip
result = happy_tt.generate_text(pass_this, args=beam_settings)
print(result.text)

Enter a sentence: He is a nice purson.
He is a nice person.


In [26]:
user_ip = input("Enter a sentence: ")
pass_this = "grammar:" + user_ip
result = happy_tt.generate_text(pass_this, args=beam_settings)
print(result.text)

Enter a sentence: Understandng
Understanding and understanding a variety of topics.


In [27]:
user_ip = input("Enter a sentence: ")
pass_this = "grammar:" + user_ip
result = happy_tt.generate_text(pass_this, args=beam_settings)
print(result.text)

Enter a sentence: Grammer
Grammer:Grammer:Grammer:Grammer:Grammer:


In [28]:
user_ip = input("Enter a sentence: ")
pass_this = "grammar:" + user_ip
result = happy_tt.generate_text(pass_this, args=beam_settings)
print(result.text)

Enter a sentence: She has do all her work
She has done all her work.


In [29]:
user_ip = input("Enter a sentence: ")
pass_this = "grammar:" + user_ip
result = happy_tt.generate_text(pass_this, args=beam_settings)
print(result.text)

Enter a sentence: Cat has four paws
Cat has four paws .


In [30]:
user_ip = input("Enter a sentence: ")
pass_this = "grammar:" + user_ip
result = happy_tt.generate_text(pass_this, args=beam_settings)
print(result.text)

Enter a sentence: Expectd
Expected


In [31]:
user_ip = input("Enter a sentence: ")
pass_this = "grammar:" + user_ip
result = happy_tt.generate_text(pass_this, args=beam_settings)
print(result.text)

Enter a sentence: His name were John. He used to live next to Anna.
His name was John and he used to live next to Anna.


In [32]:
user_ip = input("Enter a sentence: ")
pass_this = "grammar:" + user_ip
result = happy_tt.generate_text(pass_this, args=beam_settings)
print(result.text)

Enter a sentence: Please do not smok here. Its a hospitul.
Please do not smoke here, it is a friendly place.


In [33]:
user_ip = input("Enter a sentence: ")
pass_this = "grammar:" + user_ip
result = happy_tt.generate_text(pass_this, args=beam_settings)
print(result.text)

Enter a sentence: Kindly mov away from that plac
Kindly mov away from that placard


In [34]:
user_ip = input("Enter a sentence: ")
pass_this = "grammar:" + user_ip
result = happy_tt.generate_text(pass_this, args=beam_settings)
print(result.text)

Enter a sentence: This modl is working fine
This modl is working fine .


In [35]:
user_ip = input("Enter a sentence: ")
pass_this = "grammar:" + user_ip
result = happy_tt.generate_text(pass_this, args=beam_settings)
print(result.text)

Enter a sentence: Confidental
Confidential


In [36]:
user_ip = input("Enter a sentence: ")
pass_this = "grammar:" + user_ip
result = happy_tt.generate_text(pass_this, args=beam_settings)
print(result.text)

Enter a sentence: He is passed
He is passed on to the next generation .


In [37]:
user_ip = input("Enter a sentence: ")
pass_this = "grammar:" + user_ip
result = happy_tt.generate_text(pass_this, args=beam_settings)
print(result.text)

Enter a sentence: we was about to leave but he stopped us.
We were about to leave but he stopped us.


In [38]:
user_ip = input("Enter a sentence: ")
pass_this = "grammar:" + user_ip
result = happy_tt.generate_text(pass_this, args=beam_settings)
print(result.text)

Enter a sentence: Are it too stubborn looking?
Are it too stubborn looking?


In [39]:
user_ip = input("Enter a sentence: ")
pass_this = "grammar:" + user_ip
result = happy_tt.generate_text(pass_this, args=beam_settings)
print(result.text)

Enter a sentence: She waked up and chose violence.
She waked up and chose violence .


In [40]:
user_ip = input("Enter a sentence: ")
pass_this = "grammar:" + user_ip
result = happy_tt.generate_text(pass_this, args=beam_settings)
print(result.text)

Enter a sentence: He lookd lost today
He looked lost today.


In [41]:
user_ip = input("Enter a sentence: ")
pass_this = "grammar:" + user_ip
result = happy_tt.generate_text(pass_this, args=beam_settings)
print(result.text)

Enter a sentence: Is it too raini outside?
Is it too rainy outside?


In [42]:
happy_tt.save("grammar_correction/")

Configuration saved in grammar_correction/config.json
Model weights saved in grammar_correction/pytorch_model.bin
tokenizer config file saved in grammar_correction/tokenizer_config.json
Special tokens file saved in grammar_correction/special_tokens_map.json
Copy vocab file to grammar_correction/spiece.model
